In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:

import pandas as pd

ROOT_PATH = 'C:/AgodaGit/passivbot'
BACKTESTS_PATH = ROOT_PATH + '/backtests/binance'

In [9]:
import glob
import os

# list all json files in cfgs_bkup and its child folders
live_configs = glob.glob(ROOT_PATH + '/configs/live/older_versions/**/*.json', recursive=True)
# normalize json_files by replacing all \\ with /
live_configs = [f.replace('\\', '/') for f in live_configs]


#group json_files by immediate parent folder
currently_running_configs = []
for f in live_configs:
    folder = os.path.dirname(f).split('/')[-1]
    # folder looks like this : 5_9_0 or 5_9_3 or 5_9_4 etc. convert it to v5.9.0 or v.5.9.3 or v5.9.4
    version = 'v' + folder.replace('_', '.')
    currently_running_configs.append({
        'version': version,
        'production_config': f,
        'coin': os.path.basename(f).replace('.json', '')
    })

currently_running_configs

[{'version': 'v5.9.0',
  'production_config': 'C:/AgodaGit/passivbot/configs/live/older_versions/5_9_0/ATOM.json',
  'coin': 'ATOM'},
 {'version': 'v5.9.0',
  'production_config': 'C:/AgodaGit/passivbot/configs/live/older_versions/5_9_0/CHZ.json',
  'coin': 'CHZ'},
 {'version': 'v5.9.0',
  'production_config': 'C:/AgodaGit/passivbot/configs/live/older_versions/5_9_0/DOGE.json',
  'coin': 'DOGE'},
 {'version': 'v5.9.0',
  'production_config': 'C:/AgodaGit/passivbot/configs/live/older_versions/5_9_0/DOT.json',
  'coin': 'DOT'},
 {'version': 'v5.9.0',
  'production_config': 'C:/AgodaGit/passivbot/configs/live/older_versions/5_9_0/DYDX.json',
  'coin': 'DYDX'},
 {'version': 'v5.9.0',
  'production_config': 'C:/AgodaGit/passivbot/configs/live/older_versions/5_9_0/EOS.json',
  'coin': 'EOS'},
 {'version': 'v5.9.0',
  'production_config': 'C:/AgodaGit/passivbot/configs/live/older_versions/5_9_0/HBAR.json',
  'coin': 'HBAR'},
 {'version': 'v5.9.0',
  'production_config': 'C:/AgodaGit/passivbot

In [10]:
import re
'''
Result file looks like this:
    +------------------------------------------+
    |                 Summary                  |
    +-----------------------------+------------+
    | Metric                      | Value      |
    +-----------------------------+------------+
    | Exchange                    | bybit      |
    | Market type                 | futures    |
    | Symbol                      | ADAUSDT    |
    | Passivbot mode              | emas       |
    | No. days                    | 1100.0     |
    +-----------------------------+------------+
Write a parser function which takes in the path to result.txt file, reads it and then split the Metric and Value columns and put them in a dictionary and return it
'''
def parse_result_file(result_file):
    with open(result_file, 'r') as file:

        pattern = r'/v(\d+\.\d+(?:\.\w+)?)/'
        match = re.search(pattern, result_file)
        if match:
            version = match.group(1)
        else:
            version = None


        data = file.read().split('\n')
        d = {'version':version}
        current_state = None
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()
                # dont add if key is empty string
                if key == '':
                    continue

                if key == 'Long':
                    current_state = 'L'
                if key == 'Short':
                    current_state = 'S'

                # check if value can be converted to float and if so, convert it
                try:
                    value = value.replace('%', '').strip()
                    value = float(value)
                except:
                    pass

                # if current_state is not None, then add it to key with a prefix of (L) or (S)
                if current_state is not None:
                    key = f'({current_state}){key}'
                d[key] = value
    return d

# test it
parse_result_file(r'C:\AgodaGit\passivbot\backtests\binance\1INCHUSDT\plots\2023-08-20T210135\backtest_result.txt')

{'version': None,
 'Metric': 'Value',
 'Exchange': 'binance',
 'Market type': 'futures',
 'Symbol': '1INCHUSDT',
 'Passivbot mode': 'recursive_grid',
 'ADG n subdivisions': 10.0,
 'No. days': 230.0,
 'Starting balance': 1000.0,
 '(L)Long': 'True',
 '(L)ADG per exposure': -0.00493,
 '(L)ADG weighted per exposure': -0.232,
 '(L)Final balance': 988.667,
 '(L)Final equity': 782.513,
 '(L)Net PNL + fees': -11.3333,
 '(L)Net Total gain': -1.133,
 '(L)Average daily gain': -0.00493,
 '(L)Average daily gain weighted': -0.232,
 '(L)Loss to profit ratio': 1.045,
 '(L)Exposure ratios mean': 0.16409,
 '(L)Price action distance mean': 0.0239994,
 '(L)Price action distance std': 0.0429174,
 '(L)Price action distance max': 0.351351,
 '(L)Closest bankruptcy': 99.2,
 '(L)Lowest equity/balance ratio': 0.7386,
 '(L)Mean of 10 worst eq/bal ratios': 0.7502,
 '(L)Equity/balance ratio std': 0.03093,
 '(L)Ratio of time spent at max exposure': 0.1342,
 '(L)No. fills': 1162.0,
 '(L)No. entries': 632.0,
 '(L)No. 

In [14]:
import json

# backtest configurations
backtest_configurations = glob.glob(BACKTESTS_PATH + '/**/live_config.json', recursive=True)
# normalize json_files by replacing all \\ with /
backtest_configurations = [f.replace('\\', '/') for f in backtest_configurations]


def compare_key_value_of_two_json_objects(json1, json2):
    
    keys_to_ignore = ['backwards_tp','ema_dist_close','ema_dist_entry']
    
    for key in json1:
        if key in keys_to_ignore:
            continue
        if key not in json2:
            #print(f'key {key} not found in json2')
            return False
        if json1[key] != json2[key]:
            #print(f'key {key} has different value in json2')
            return False
    return True


# now iterate over config_by_version and find the downloaded config.json file for each live config

for live_config in currently_running_configs:
    version = live_config['version']
    coin = live_config['coin'] + 'USDT'
    production_config_json = json.load(open(live_config['production_config']))
    # find the list of matches in downloaded config.json files for this coin where version must be in the path and coin must be in the path
    matches = [f for f in backtest_configurations if coin in f]

    match_found = None

    for match in matches:
        live_config_json = json.load(open(match))

        # the live_config_json["long"] and live_config_json["short"] matches file_content['long'] and file_content['short'] respectively, then we have a match
        if compare_key_value_of_two_json_objects(live_config_json["long"], production_config_json['long']) and compare_key_value_of_two_json_objects(
                live_config_json["short"], production_config_json['short']):
            match_found = match
            break

    if match_found is None:
        print(f'No match found for {coin} in {version}')
    else:
        live_config['backtest_config'] = match_found
        # find the backtest result file ( called result.txt ) for this downloaded config.json file. it is found in a subdirectory within the same folder as config.json
        #print(glob.glob(os.path.dirname(match_found)))
        result_file = glob.glob(os.path.dirname(match_found) + '/**/backtest_result.txt', recursive=True)
        live_config['result_file'] = result_file[0].replace('\\', '/') if len(result_file) > 0 else None
        live_config['result'] = parse_result_file(live_config['result_file']) if live_config[
                                                                                     'result_file'] is not None else None
        live_config['result']['version'] = version

currently_running_configs

[{'version': 'v5.9.0',
  'production_config': 'C:/AgodaGit/passivbot/configs/live/older_versions/5_9_0/ATOM.json',
  'coin': 'ATOM',
  'backtest_config': 'C:/AgodaGit/passivbot/backtests/binance/ATOMUSDT/plots/2023-08-20T223800/live_config.json',
  'result_file': 'C:/AgodaGit/passivbot/backtests/binance/ATOMUSDT/plots/2023-08-20T223800/backtest_result.txt',
  'result': {'version': 'v5.9.0',
   'Metric': 'Value',
   'Exchange': 'binance',
   'Market type': 'futures',
   'Symbol': 'ATOMUSDT',
   'Passivbot mode': 'recursive_grid',
   'ADG n subdivisions': 10.0,
   'No. days': 170.0,
   'Starting balance': 1000.0,
   '(L)Long': 'True',
   '(L)ADG per exposure': 0.0792,
   '(L)ADG weighted per exposure': 0.116,
   '(L)Final balance': 1034.61,
   '(L)Final equity': 1035.11,
   '(L)Net PNL + fees': 35.1217,
   '(L)Net Total gain': 3.461,
   '(L)Average daily gain': 0.0198,
   '(L)Average daily gain weighted': 0.0289,
   '(L)Loss to profit ratio': 0.5152,
   '(L)Exposure ratios mean': 0.26873

In [15]:
'''
Now iterate over currently_running_configs and add the result's key/value to a dataframe
[{'version': 'v5.9.0',
  'production_config': 'C:/AgodaGit/passivbot/configs/live/older_versions/5_9_0/ATOM.json',
  'coin': 'ATOM',
  'backtest_config': 'C:/AgodaGit/passivbot/backtests/binance/ATOMUSDT/plots/2023-08-20T181841/live_config.json',
  'result_file': 'C:/AgodaGit/passivbot/backtests/binance/ATOMUSDT/plots/2023-08-20T181841/backtest_result.txt',
  'result': {'version': 'v5.9.0',... }}, 
 ]  
'''

currently_running_df = pd.DataFrame()
for live_config in currently_running_configs:
    if live_config['result'] is None:
        continue
    # add the result's key/value to a dataframe
    for k, v in live_config['result'].items():
        currently_running_df.loc[live_config['coin'], k] = v
    
    #all add a column for result_file and backtest_config
    currently_running_df.loc[live_config['coin'], 'result_file'] = live_config['result_file']
    currently_running_df.loc[live_config['coin'], 'backtest_config'] = live_config['backtest_config']

# drop Metric , Exchange , Market type
currently_running_df = currently_running_df.drop(['Metric', 'Exchange', 'Market type'], axis=1)

currently_running_df



,version,Symbol,Passivbot mode,ADG n subdivisions,No. days,Starting balance,(L)Long,(L)ADG per exposure,(L)ADG weighted per exposure,(L)Final balance,...,(S)Average n fills per day,(S)Mean hours stuck,(S)Max hours stuck,(S)PNL sum,(S)Profit sum,(S)Loss sum,(S)Fee sum,(S)Volume quote,result_file,backtest_config
ATOM,v5.9.0,ATOMUSDT,recursive_grid,10.0,170.0,1000.0,True,0.07920,0.11600,1034.610,...,3.75,6.31100,194.9500,16.700,19.89,-3.189,-1.2250,8341.44,C:/AgodaGit/passivbot/backtests/binance/ATOMUS...,C:/AgodaGit/passivbot/backtests/binance/ATOMUS...
CHZ,v5.9.0,CHZUSDT,recursive_grid,10.0,170.0,1000.0,True,-0.11500,-0.20200,951.560,...,7.69,3.09830,194.9000,39.480,43.67,-4.190,-2.7490,20466.70,C:/AgodaGit/passivbot/backtests/binance/CHZUSD...,C:/AgodaGit/passivbot/backtests/binance/CHZUSD...
DOGE,v5.9.0,DOGEUSDT,recursive_grid,10.0,170.0,1000.0,True,0.06330,-0.20100,1027.600,...,2.47,9.67155,316.3830,26.370,35.41,-9.037,-1.2340,14135.50,C:/AgodaGit/passivbot/backtests/binance/DOGEUS...,C:/AgodaGit/passivbot/backtests/binance/DOGEUS...
DOT,v5.9.0,DOTUSDT,recursive_grid,10.0,170.0,1000.0,True,0.08430,0.13500,1036.890,...,5.79,4.12886,114.4170,-1.571,19.54,-21.110,-1.5500,18044.60,C:/AgodaGit/passivbot/backtests/binance/DOTUSD...,C:/AgodaGit/passivbot/backtests/binance/DOTUSD...
DYDX,v5.9.0,DYDXUSDT,recursive_grid,10.0,170.0,1000.0,True,0.06350,0.10700,1027.670,...,21.00,1.13315,61.9167,13.910,149.60,-135.600,-7.1840,205037.00,C:/AgodaGit/passivbot/backtests/binance/DYDXUS...,C:/AgodaGit/passivbot/backtests/binance/DYDXUS...
EOS,v5.9.0,EOSUSDT,recursive_grid,10.0,170.0,1000.0,True,-0.02320,0.09980,990.093,...,6.29,3.77332,195.5500,30.670,43.23,-12.550,-2.7220,25917.40,C:/AgodaGit/passivbot/backtests/binance/EOSUSD...,C:/AgodaGit/passivbot/backtests/binance/EOSUSD...
HBAR,v5.9.0,HBARUSDT,recursive_grid,10.0,170.0,1000.0,True,0.07250,0.37500,1031.640,...,6.91,3.46341,157.1670,25.310,47.31,-22.000,-3.2240,32391.50,C:/AgodaGit/passivbot/backtests/binance/HBARUS...,C:/AgodaGit/passivbot/backtests/binance/HBARUS...
LINK,v5.9.0,LINKUSDT,recursive_grid,10.0,170.0,1000.0,True,-0.11300,0.00794,952.694,...,3.18,7.48141,165.4500,15.410,16.93,-1.519,-0.8366,6271.05,C:/AgodaGit/passivbot/backtests/binance/LINKUS...,C:/AgodaGit/passivbot/backtests/binance/LINKUS...
MANA,v5.9.0,MANAUSDT,recursive_grid,10.0,170.0,1000.0,True,-0.07780,0.09660,967.090,...,4.12,5.75311,184.3330,38.720,49.12,-10.400,-1.3930,18969.30,C:/AgodaGit/passivbot/backtests/binance/MANAUS...,C:/AgodaGit/passivbot/backtests/binance/MANAUS...
MATIC,v5.9.0,MATICUSDT,recursive_grid,10.0,170.0,1000.0,True,-0.01550,-0.21900,993.367,...,5.29,4.52506,231.3830,32.600,36.52,-3.914,-1.5550,9310.95,C:/AgodaGit/passivbot/backtests/binance/MATICU...,C:/AgodaGit/passivbot/backtests/binance/MATICU...


In [20]:
# iterate over the backtest configuration , find result.txt file and parse it and add it to a dataframe

import pandas as pd

# backtest configurations
backtest_configurations = glob.glob(BACKTESTS_PATH + '/**/live_config.json', recursive=True)
# normalize json_files by replacing all \\ with /
backtest_configurations = [f.replace('\\', '/') for f in backtest_configurations]

# create a set from bactest_config column of currently_running_df
l_config = set(currently_running_df['backtest_config'].values)

result_analysis = pd.DataFrame()
for backtest_config in backtest_configurations:
    result_file = os.path.dirname(backtest_config) + '/backtest_result.txt'
    if not os.path.exists(result_file):
        continue
    #print('result_file file : ', result_file )
    result = parse_result_file(result_file)
    result['file'] = backtest_config
    if backtest_config not in l_config:
        result['live'] = False
    else:
        result['live'] = True
    # convert result to a pandas dataframe and append it to result_analysis dataframe
    result = pd.DataFrame(result, index=[0])
    result_analysis = pd.concat([result_analysis, result], ignore_index=True)

result_analysis = result_analysis.drop(['version', 'Metric', 'Exchange', 'Market type'], axis=1)

result_analysis



,Symbol,Passivbot mode,ADG n subdivisions,No. days,Starting balance,(L)Long,(L)ADG per exposure,(L)ADG weighted per exposure,(L)Final balance,(L)Final equity,...,(S)Average n fills per day,(S)Mean hours stuck,(S)Max hours stuck,(S)PNL sum,(S)Profit sum,(S)Loss sum,(S)Fee sum,(S)Volume quote,file,live
0,1000LUNCUSDT,recursive_grid,10.0,230.0,1000.0,True,0.19900,0.08720,1582.930,1449.690,...,4.30,5.55557,83.1500,128.4,128.4,0.000,-4.291,33265.5,C:/AgodaGit/passivbot/backtests/binance/1000LU...,False
1,1000LUNCUSDT,clock,10.0,170.0,1000.0,True,0.11800,0.12100,1225.410,1225.260,...,7.96,3.01088,25.6833,229.5,229.5,0.000,-11.570,144471.0,C:/AgodaGit/passivbot/backtests/binance/1000LU...,False
2,1000LUNCUSDT,clock,10.0,170.0,1000.0,True,0.09060,0.07660,1168.600,1168.600,...,8.54,2.80803,51.4000,268.1,280.5,-12.390,-10.270,133200.0,C:/AgodaGit/passivbot/backtests/binance/1000LU...,False
3,1000LUNCUSDT,clock,10.0,170.0,1000.0,True,0.11500,0.08590,1219.210,1219.210,...,8.55,2.80421,51.4000,268.4,281.0,-12.580,-10.280,133239.0,C:/AgodaGit/passivbot/backtests/binance/1000LU...,False
4,1000LUNCUSDT,clock,10.0,170.0,1000.0,True,0.09120,-0.00936,1169.810,979.935,...,7.37,3.25188,50.0167,238.3,239.8,-1.519,-6.436,80944.9,C:/AgodaGit/passivbot/backtests/binance/1000LU...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8258,ZRXUSDT,neat_grid,10.0,170.0,1000.0,True,0.04330,-0.05470,1077.260,1077.250,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C:/AgodaGit/passivbot/backtests/binance/ZRXUSD...,False
8259,ZRXUSDT,neat_grid,10.0,170.0,1000.0,True,-0.08600,-0.03800,970.852,964.496,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C:/AgodaGit/passivbot/backtests/binance/ZRXUSD...,False
8260,ZRXUSDT,neat_grid,10.0,170.0,1000.0,True,0.00772,0.05090,1013.370,1013.370,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C:/AgodaGit/passivbot/backtests/binance/ZRXUSD...,False
8261,ZRXUSDT,neat_grid,10.0,170.0,1000.0,True,-0.04860,-0.05720,983.429,974.989,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C:/AgodaGit/passivbot/backtests/binance/ZRXUSD...,False


In [27]:
def shorten_string(string):
    # replace everything between parentheses
    string = re.sub(r'\([^)]*\)', '', string)

    # create a dictionary of key value to replace in string
    replace_dict = {'ADG per exposure': 'adg_per_exp',
                    'ADG weighted per exposure': 'adg_w_per_exp',
                    'Average daily gain': 'adg',
                    'Average daily gain weighted': 'adg_w',
                    'Closest bankruptcy': 'bnkr',
                    'Equity/balance ratio std': 'ebr_std',
                    'Exposure ratios mean': 'exp_rat_mean',
                    'Loss to profit ratio': 'l2p_r',
                    'Lowest equity/balance ratio': 'l_ebr',
                    'Max hours stuck': 'max_stuck',
                    'Mean hours stuck': 'mean_stuck',
                    'Mean of 10 worst eq/bal ratios': 'mean_10_ebr',
                    'No. initial entries': 'n_init_ent',
                    'No. normal closes': 'n_norm_close',
                    'No. reentries': 'n_re_ent',
                    'No. unstuck closes': 'n_unstk_close',
                    'No. unstuck entries': 'n_unstk_ent',
                    'Price action distance max': 'pa_d_max',
                    'Price action distance mean': 'pa_d_mean',
                    'Price action distance std': 'pa_d_std',
                    'ADG n subdivisions': 'adg_n_subdiv',
                    'Starting balance': 'start_bal',
                    'Final balance': 'final_bal',
                    'No. unstuck/EMA closes': 'n_unstk_ema_close',
                    'Average n fills per day': 'avg_n_fills_per_day',
                    'PNL sum': 'pnl_sum',
                    'Profit sum': 'profit_sum',
                    'Loss sum': 'loss_sum',
                    'Fee sum': 'fee_sum',
                    'Volume quote': 'vol_quote',
                    'Exchange': 'exch',
                    'No. days': 'n_days',
                    'Passivbot mode': 'mode',
                    'Symbol': 's',
                    'file': 'f',
                    'live': 'live',
                    'min_notional': 'min_notion',
                    'version': 'ver',
                    'volume': 'vol'}

    # find the longest key match in replace_dict and replace it with its value
    for k in sorted(replace_dict, key=len, reverse=True):
        string = string.replace(k, replace_dict[k])

    return string


'''
call shorten_string function with the followings and print result:
 '(L)ADG per exposure'
 '(L)ADG weighted per exposure'
 '(L)Average daily gain'
 '(L)Average daily gain weighted'
'''
print(shorten_string('(L)ADG per exposure'))
print(shorten_string('(L)ADG weighted per exposure'))
print(shorten_string('(L)Average daily gain'))
print(shorten_string('(L)Average daily gain weighted'))

# filter now
#result_analysis = big_data[(big_data.s.isin(allowed_symbols))
#display(result_analysis.columns)

# create result_analysis_long by dropping all columns which has (S) in it
result_analysis_long = result_analysis[[c for c in result_analysis.columns if '(S)' not in c]]
# create result_analysis_short by dropping all columns which has (L) in it
result_analysis_short = result_analysis[[c for c in result_analysis.columns if '(L)' not in c]]

# replace  (S) and (L) in all column names with empty string
result_analysis_long.columns = result_analysis_long.columns.str.replace(r'\(S\)|\(L\)', '')
result_analysis_short.columns = result_analysis_short.columns.str.replace(r'\(S\)|\(L\)', '')

display(result_analysis_long.columns)
result_analysis_long = result_analysis_long[
    (result_analysis_long['live'] == 'True') |
    (
            (result_analysis_long['No. days'] == 170) &
            (result_analysis_long['Average daily gain weighted'] > 0 ) &
            (result_analysis_long['ADG weighted per exposure'] > 0 ) &
            (result_analysis_long['Final balance'] >  result_analysis_long['Starting balance'] * 1.05) &
            (result_analysis_long['Final equity'] >  result_analysis_long['Starting balance']) &
            (result_analysis_long['Lowest equity/balance ratio'] > 0.92 ) &
            (result_analysis_long['Mean of 10 worst eq/bal ratios'] > 0.95 ) &
            (result_analysis_long['Loss to profit ratio'] != 0 )
    )
    ]


result_analysis_short = result_analysis_short[
    (result_analysis_short['live'] == 'True') |
    (
            (result_analysis_short['No. days'] == 170) &
            (result_analysis_short['Average daily gain weighted'] > 0 ) &
            (result_analysis_short['ADG weighted per exposure'] > 0 ) &
            (result_analysis_short['Final balance'] >  result_analysis_short['Starting balance'] * 1.05) &
            (result_analysis_short['Final equity'] >  result_analysis_short['Starting balance']) &
            (result_analysis_short['Lowest equity/balance ratio'] > 0.92 ) &
            (result_analysis_short['Mean of 10 worst eq/bal ratios'] > 0.95 ) &
            (result_analysis_short['Loss to profit ratio'] != 0 )
    )
    ]

result_analysis_long = result_analysis_long.rename(columns=shorten_string)
result_analysis_short = result_analysis_short.rename(columns=shorten_string)

display(result_analysis_long, result_analysis_short)

# dump it in an excel for analysis
EXCEL_FILE_NAME = r'C:\AgodaGit\passivbot\analysis_scripts\backtest_result.xlsx'
excel_writer = pd.ExcelWriter(EXCEL_FILE_NAME, engine='xlsxwriter')
result_analysis_long.to_excel(excel_writer, index=False, sheet_name='long')
result_analysis_short.to_excel(excel_writer, index=False, sheet_name='short')
excel_writer.save()
excel_writer.close()


adg_per_exp
adg_w_per_exp
adg
adg_w


C:\Users\skhan\AppData\Local\Temp\ipykernel_19160\798128456.py:75: FutureWarning: The default value of regex will change from True to False in a future version.
  result_analysis_long.columns = result_analysis_long.columns.str.replace(r'\(S\)|\(L\)', '')
C:\Users\skhan\AppData\Local\Temp\ipykernel_19160\798128456.py:76: FutureWarning: The default value of regex will change from True to False in a future version.
  result_analysis_short.columns = result_analysis_short.columns.str.replace(r'\(S\)|\(L\)', '')


Index(['Symbol', 'Passivbot mode', 'ADG n subdivisions', 'No. days',
       'Starting balance', 'Long', 'ADG per exposure',
       'ADG weighted per exposure', 'Final balance', 'Final equity',
       'Net PNL + fees', 'Net Total gain', 'Average daily gain',
       'Average daily gain weighted', 'Loss to profit ratio',
       'Exposure ratios mean', 'Price action distance mean',
       'Price action distance std', 'Price action distance max',
       'Closest bankruptcy', 'Lowest equity/balance ratio',
       'Mean of 10 worst eq/bal ratios', 'Equity/balance ratio std',
       'Ratio of time spent at max exposure', 'No. fills', 'No. entries',
       'No. closes', 'No. initial entries', 'No. reentries',
       'No. unstuck/EMA entries', 'No. unstuck/EMA closes',
       'No. normal closes', 'Average n fills per day', 'Mean hours stuck',
       'Max hours stuck', 'PNL sum', 'Profit sum', 'Loss sum', 'Fee sum',
       'Volume quote', 'file', 'live'],
      dtype='object')

,s,mode,adg_n_subdiv,n_days,start_bal,Long,adg_per_exp,adg_w_per_exp,final_bal,Final equity,...,avg_n_fills_per_day,mean_stuck,max_stuck,pnl_sum,profit_sum,loss_sum,fee_sum,vol_quote,f,live
2,1000LUNCUSDT,clock,10.0,170.0,1000.0,True,0.0906,0.0766,1168.60,1168.60,...,4.67,5.12741,29.9667,174.50,181.30,-6.7760,-5.902,31477.8,C:/AgodaGit/passivbot/backtests/binance/1000LU...,False
12,1000SHIBUSDT,clock,10.0,170.0,1000.0,True,0.0987,0.1330,1184.87,1184.81,...,5.69,4.21397,50.8333,193.40,193.60,-0.2109,-8.566,75317.1,C:/AgodaGit/passivbot/backtests/binance/1000SH...,False
13,1000SHIBUSDT,clock,10.0,170.0,1000.0,True,0.1050,0.1160,1197.82,1197.58,...,10.80,2.22644,28.5833,215.30,216.60,-1.2830,-17.480,76548.6,C:/AgodaGit/passivbot/backtests/binance/1000SH...,False
14,1000SHIBUSDT,clock,10.0,170.0,1000.0,True,0.1010,0.1350,1189.94,1189.87,...,5.73,4.18828,50.9000,198.70,198.90,-0.2074,-8.790,78181.2,C:/AgodaGit/passivbot/backtests/binance/1000SH...,False
15,1000SHIBUSDT,clock,10.0,170.0,1000.0,True,0.1010,0.1340,1189.23,1189.17,...,5.73,4.18828,50.9000,198.00,198.20,-0.2057,-8.762,78073.6,C:/AgodaGit/passivbot/backtests/binance/1000SH...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7774,XRPUSDT,recursive_grid,10.0,170.0,1000.0,True,0.1730,0.1240,1061.27,1053.93,...,10.20,2.32861,62.6000,64.97,72.22,-7.2550,-3.701,42281.8,C:/AgodaGit/passivbot/backtests/binance/XRPUSD...,False
8114,ZILUSDT,clock,10.0,170.0,1000.0,True,0.1440,0.0657,1077.05,1077.05,...,8.97,2.65854,67.4000,80.41,91.31,-10.9100,-3.354,49611.7,C:/AgodaGit/passivbot/backtests/binance/ZILUSD...,False
8172,ZRXUSDT,neat_grid,10.0,170.0,1000.0,True,0.0780,0.0565,1143.47,1143.47,...,6.59,3.62147,51.5667,149.70,150.00,-0.3370,-6.230,61588.8,C:/AgodaGit/passivbot/backtests/binance/ZRXUSD...,False
8174,ZRXUSDT,recursive_grid,10.0,170.0,1000.0,True,0.1050,0.0727,1197.15,1197.15,...,5.83,4.09034,49.7500,206.00,206.40,-0.4058,-8.864,48970.3,C:/AgodaGit/passivbot/backtests/binance/ZRXUSD...,False


,s,mode,adg_n_subdiv,n_days,start_bal,Short,adg_per_exp,adg_w_per_exp,final_bal,Final equity,...,avg_n_fills_per_day,mean_stuck,max_stuck,pnl_sum,profit_sum,loss_sum,fee_sum,vol_quote,f,live
5,1000LUNCUSDT,clock,10.0,170.0,1000.0,True,0.1050,0.1250,1197.90,1194.91,...,12.20,1.95998,35.4833,205.3,208.3,-3.0620,-7.371,243740.0,C:/AgodaGit/passivbot/backtests/binance/1000LU...,False
13,1000SHIBUSDT,clock,10.0,170.0,1000.0,True,0.1180,0.1370,1224.13,1224.11,...,5.31,4.52571,36.9667,234.2,238.8,-4.5860,-10.050,91426.0,C:/AgodaGit/passivbot/backtests/binance/1000SH...,False
143,ACHUSDT,clock,10.0,170.0,1000.0,True,0.2090,0.2230,1432.76,1422.42,...,14.90,1.61618,25.8500,455.1,463.9,-8.8010,-22.340,156855.0,C:/AgodaGit/passivbot/backtests/binance/ACHUSD...,False
144,ACHUSDT,clock,10.0,170.0,1000.0,True,0.2090,0.2230,1433.06,1422.38,...,14.90,1.61614,25.8500,455.5,464.3,-8.8250,-22.410,155020.0,C:/AgodaGit/passivbot/backtests/binance/ACHUSD...,False
146,ACHUSDT,clock,10.0,170.0,1000.0,True,0.1750,0.1840,1351.93,1341.38,...,18.40,1.30288,41.8000,374.9,375.6,-0.7089,-22.990,189364.0,C:/AgodaGit/passivbot/backtests/binance/ACHUSD...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8066,ZILUSDT,clock,10.0,170.0,1000.0,True,0.0917,0.0642,1170.83,1167.59,...,18.30,1.30967,23.2333,188.1,190.8,-2.6880,-17.250,159066.0,C:/AgodaGit/passivbot/backtests/binance/ZILUSD...,False
8067,ZILUSDT,clock,10.0,170.0,1000.0,True,0.0925,0.0673,1172.36,1169.11,...,18.30,1.31468,23.2333,189.8,192.7,-2.8510,-17.450,162319.0,C:/AgodaGit/passivbot/backtests/binance/ZILUSD...,False
8069,ZILUSDT,neat_grid,10.0,170.0,1000.0,True,0.0591,0.0314,1106.89,1106.60,...,13.20,1.81715,56.6833,116.9,117.7,-0.7795,-10.040,76094.7,C:/AgodaGit/passivbot/backtests/binance/ZILUSD...,False
8070,ZILUSDT,recursive_grid,10.0,170.0,1000.0,True,0.0552,0.0197,1099.55,1099.46,...,6.86,3.46234,89.0167,105.8,106.4,-0.6252,-6.227,44736.6,C:/AgodaGit/passivbot/backtests/binance/ZILUSD...,False


C:\Users\skhan\.virtualenvs\passivbot\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
